## Access to the World Database on Protected Areas (WDPA) historical data and harmonization

This block is dedicated to refining initial land-use/land-cover (LULC) data with additional data on protected areas (PA) from [the World Database on Protected Areas (WDPA)](https://www.protectedplanet.net/en/thematic-areas/wdpa).
As soon as protected areas may significantly reduce the reflectance of landscapes for species migration, landscapes intersected with PAs should be considered as different from those with no protected status. This workflow is describing the process of updating LULC data needed to compute functional landscape connectivity. It provides two main outputs:
- LULC data enriched with protected areas (recorded as updated LULC value) for wide usage.
- For habitat connectivity calculations, impedance and affinity values for calculations in specific  software (Miramon and Graphab).

Current limitations:
- WDPA API is accessed through personal credentials, while granting access to the API is not automatic and reviewed by the Protected Planet team.
- WDPA API does not support getting data by bounding box, only by unique IDs of protected areas and countries.
- Temporary server outage has been experienced with WDPA API (returning 'status code 500').
- If a protected area is deestablished ('degazetted'), it is removed from the database and its ID cannot be reused (for further details, see the [manual on WDPA API](https://wdpa.s3-eu-west-1.amazonaws.com/WDPA_Manual/English/WDPA_WDOECM_Manual_1_6.pdf). If it is the case, all historical transformations of these protected areas will be not accessible to request.
- Nominatim API is used an ancillary tool to find countries intersecting with the input raster dataset to query for data through WDPA API. However, geometrical inconsistency may persist if boundaries. At the same time, boundaries of countries include the exclusive economic zones in seas and can cover not only terrestrial protected areas.

#### 1. Extracting data through WDPA API

Spatial data on protected areas in GeoJSON and GeoPackage formats for countries needed (on our case, Spain, France and Andorra) are obtained through WDPA API using a personal access token and [official docimentation](https://api.protectedplanet.net/documentation). Most meaningful attributes have been chosen (IDs, designation status, IUCN category, year of establishment etc.)

Let's import libraries neeeded:

In [1]:
import requests
from shapely.geometry import shape
import json
import subprocess
import os
from datetime import datetime

import yaml

# own modules
import timing # TODO - to define it from another directory

Input variables are stored in the configuration file (eg input raster dataset, timestamp). Let's read them:

In [2]:
# define current directory
current_dir = os.getcwd()

"""
# This method doesn't work in Jupyter Notebook
# define current directory
current_dir = os.path.dirname(os.path.abspath(__file__))
# os.path.abspath(__file__) is not defined in interactive environments (Jupyter Notebooks)
"""

# define path to the configuration file (one level above)
config_path = os.path.join(current_dir, '..', 'config.yaml')
# open config file
with open(config_path,'r') as f:
    config = yaml.safe_load(f)

# read yearstamp from the configuration file
year = config.get('year')
if year is None or 'year' not in config: # both conditions should be considered
    warnings.warn("Year variable is not found in the configuration file.")

# define input raster dataset to be enriched with data on protected areas
lulc_template = config.get('lulc')
# substitute year from the configuration file
lulc_file = lulc_template.format(year=year)

# define path to input raster dataset
lulc_dir = config.get('lulc_dir')
lulc = os.path.join(current_dir, '..', lulc_dir, lulc_file) # to spot lulc_file at the level above

# normalise the path to ensure it is correctly formatted
lulc = os.path.normpath(lulc)

print(f"Input raster to be used for processing is {lulc}.")

Input raster to be used for processing is C:\Users\kriukovv\Documents\pilot_2\preprocessing\data\input\lulc\lulc_2022.tif.


Let's retrieve the bounding box of input raster dataset:

In [3]:
# TODO - cast to the common function
from osgeo import gdal
from shapely.geometry import box 

inp_source = gdal.Open(lulc)
geotransform = inp_source.GetGeoTransform()

# fetch spatial resolution
xres = geotransform[1]
yres = geotransform[5]
cell_size = abs(xres)

# fetch max/min coordinates
x_min = geotransform[0]
y_max = geotransform[3]
x_max = x_min + geotransform[1] * inp_source.RasterXSize
y_min = y_max + geotransform[5] * inp_source.RasterYSize

# define bbox through shapely library
bbox = box(x_min, y_min, x_max, y_max)

print (f"Spatial resolution (pixel size) is {cell_size} meters")
print (f"x min coordinate is {x_min}")
print (f"y max coordinate is {y_max}")
print (f"x max coordinate is {x_max}")
print (f"y min coordinate is {y_min}")
print (f"Bounding box of input raster dataset is {bbox}")

Spatial resolution (pixel size) is 30.0 meters
x min coordinate is 230205.0
y max coordinate is 4777335.0
x max coordinate is 556485.0
y min coordinate is 4459725.0
Bounding box of input raster dataset is POLYGON ((556485 4459725, 556485 4777335, 230205 4777335, 230205 4459725, 556485 4459725))


C:\Users\kriukovv\.conda\envs\overpass\Lib\site-packages\osgeo\gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


##### 1.1. Reverse geocoding
To run WDPA API it is requred to list countries for query on protected areas. Currently this is implemented through Overpass API bringing codes of countries (according  to ISO3 standard).
Other ways attempted:
- Nominatim API is unstable when quering with multiple filters (does not bring features needed).
- geopandas built-in datasets, but the dataset with the boundaries of countries is not curently available there.
- pygeoboundaries library seems to be broken.

In [5]:
import sys

import geopandas as gpd
from shapely.geometry import Polygon
import json

# add the parent directory to the Python path temporarily
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# import the reprojection script as a module
import reprojection  # this imports the reprojection.py file

# call the function from the reprojection module and bring the coordinates
cell_size, x_min, y_min, x_max, y_max = reprojection.bbox_to_WGS84(config) # TODO - delete redundant bbox

# define bbox
bbox=f"{y_min},{x_min},{y_max},{x_max}"

# TODO - to fix messed coordinates in function
print(bbox)
print('-' * 40)

# construct Overpass Turbo query to fetch countries in the bounding box
query_countries = f"""
[out:json]
[maxsize:1073741824]
[timeout:9000]
[bbox:{bbox}];
nwr["boundary"="administrative"]["admin_level"="2"]["ISO3166-1:alpha3"~"^.+$"];
/*relations must be imported as well to deliver the consistent attributes. ISO3 code couldn't be null.*/
(._;>;);
out;
"""

import os

# Overpass endpoint
overpass_url = "https://overpass-api.de/api/interpreter"

response = requests.get(overpass_url, params={'data': query_countries})
        
# if response is successful
if response.status_code == 200:
    print(f"Query to fetch OSM data for the boundaries of countries has been successful.")
    data = response.json()

    # filter for non-empty ISO3166-1:alpha3
    elements = data['elements']
    filtered_elements = [
        elem for elem in elements 
        if 'tags' in elem and 'ISO3166-1:alpha3' in elem['tags'] and elem['tags']['ISO3166-1:alpha3']
    ]
        
    # extract unique ISO3166-1:alpha3 values
    countries = {elem['tags']['ISO3166-1:alpha3'] for elem in filtered_elements}

    # print all unique ISO3166-1:alpha3 codes
    print("Unique ISO3166-1:alpha3 codes of countries within the bounding box of the input raster dataset:")
    for code in sorted(countries):
        print(code)
        
else:
    print(f"Error: {response.status_code} for fetching countries.")
    print(response.text)
    print ("-" * 30)


"""
north, south, east, west = bbox.bounds
print (north, south, east, west)
"""

# experiments with Nominatim API (redundant)
"""
import osmnx as ox
# function to bring the boundaries of countries from the bounding box
def fetch_admin_boundaries_from_bbox(bbox):
    bbox_str = ','.join(map(str, bbox))
    nominatim_url = (
        f"https://nominatim.openstreetmap.org/search?"
        f"boundary=administrative"
        f"&admin_level=2"
        f"&format=json"
        f"&bbox={bbox_str}"
    )
    
    response = requests.get(nominatim_url)
    data = response.json()
    
    # Process the data as needed
    return data

# Fetch administrative boundaries
admin_boundaries = fetch_admin_boundaries_from_bbox(bbox)
print(admin_boundaries)
"""

"""
# another function to bring boundaries through Nominatim API
import osmnx as ox

def fetch_admin_boundaries_from_bbox(bbox):
    
    # define tags to filter for countries (administrative boundaries at admin_level=2)
    tags = {'boundary': 'administrative', 'admin_level': '2'}
    
    # fetch administrative boundaries from Open Street Map within the bounding box
    gdf = ox.features_from_bbox(*bbox, tags=tags) # * is used to unpack tuple with separate arguments

    # to convert strings into lists:
    for column in gdf.columns:
        if gdf[column].apply(lambda x: isinstance(x, list)).any():
            gdf[column] = gdf[column].apply(lambda x: ','.join(map(str, x)) if isinstance(x, list) else x)
    
    print (gdf.columns)
    return gdf

def save_to_gpkg(gdf, filename='admin_boundaries.gpkg'):
    # save geojson to gpkg
    gdf.to_file(filename, driver='GPKG')

# fetch administrative boundaries
admin_boundaries_gdf = fetch_admin_boundaries_from_bbox(bbox)

# save to GeoJSON file
save_to_gpkg(admin_boundaries_gdf)
"""

"""
import pygeoboundaries

# fetch boundaries of countries
boundaries = pygeoboundaries.get_boundaries('countries', format='geojson') # level=0 for countries

# convert geojson to geodataframe, extracting features from the dataset
world = gpd.GeoDataFrame.from_features(boundaries['features'])

# find countries intersecting with bounding box
countries = world[world.geometry.intersects(bbox)]

iso3_codes = countries['iso_a3'].tolist()

print(iso3_codes)
"""

"""
import geodatasets

print(geodatasets.data)

# extract file with the boundaries of countries
world_path = geodatasets.get_path('naturalearth.countries')
world = gpd.read_file(world_path)

# find country or countries which intersect with the bounding box
countries = world[world.geometry.intersects(bbox)]

# extract ISO3 codes of countries
iso3_codes = countries['iso_a3'].tolist()

print(f"ISO3 codes of countries intersecting with the bounding box of input raster dataset: {iso3_codes}")
"""


C:\Users\kriukovv\Documents\pilot_2\preprocessing\data\input\lulc\lulc_2022.tif
Input raster dataset <osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000001FBE08223A0> > was opened successfully.
Coordinate reference system of the input raster dataset is EPSG:25831
Before reprojection:
x_min: 230205.0
x_max: 556485.0
y_min: 4459725.0
y_max: 4777335.0
After reprojection:
x_min: -0.17175642211838113
x_max: 3.6946497219995114
y_min: 40.24452664923259
y_max: 43.146655333714754
Bounding box: -0.17175642211838113,40.24452664923259,3.6946497219995114,43.146655333714754
40.24452664923259,-0.17175642211838113,43.146655333714754,3.6946497219995114
<Response [200]>
Query to fetch OSM data for the boundaries of countries has been successful.
Unique ISO3166-1:alpha3 codes:
AND
ESP
FRA


'\nimport geodatasets\n\nprint(geodatasets.data)\n\n# extract file with the boundaries of countries\nworld_path = geodatasets.get_path(\'naturalearth.countries\')\nworld = gpd.read_file(world_path)\n\n# find country or countries which intersect with the bounding box\ncountries = world[world.geometry.intersects(bbox)]\n\n# extract ISO3 codes of countries\niso3_codes = countries[\'iso_a3\'].tolist()\n\nprint(f"ISO3 codes of countries intersecting with the bounding box of input raster dataset: {iso3_codes}")\n'

##### 1.1. Building API request

In [ ]:
# getting variables from the configuration file
marine = config.get('marine') # fetch boolean value (false or true)

# define the API endpoint - include filter by country, avoid marine areas, maximum values of protected areas per page (50)
api_url = "https://api.protectedplanet.net/v3/protected_areas/search?token={token}&country={country}&marine={marine}&with_geometry=true&per_page=50"
# define token - replace by own
token = "968cef6f0c37b925225fb60ac8deaca6" 
# define country codes
countries = ["ESP", "FRA", "AND"]

# directory to save GeoJSON files
response_dir = "response"
os.makedirs(response_dir, exist_ok=True)
# list to store the names of the GeoJSON files
geojson_files = []

# TODO - country codes should derive from the extent of buffered LULC data - see section 2. It would be better to unify it, to create a separate function and apply it for all Notebooks

##### 1.2. Looping over countries

In [ ]:
# loop over each ISO code
for country in countries:
    # make GET request to the WDPA API
    url = api_url.format(country=country, token=token, marine=marine) # TODO - to include marine=marine
    response = requests.get(url)
    '''
    # to check content
    # print(response.content)
    '''
    
    # check if the request was successful
    if response.status_code == 200:
        # extract protected areas if they exist in the response
        response_json = response.json()
        protected_areas = response_json.get('protected_areas', [])

        # create GeoJSON feature collection
        feature_collection = {
            "type": "FeatureCollection",
            "features": []
        }

        # loop over protected areas        
        for pa in protected_areas:

            # convert date string to datetime object
            date_str = pa.get('legal_status_updated_at')

            # filter out protected areas if no date of establishment year is recorded
            if date_str:
                try:
                    date_obj = datetime.strptime(date_str, "%d/%m/%Y")
                    formatted_date = date_obj.strftime("%Y-%m-%d")
                except ValueError:
                    formatted_date = None
            else:
                formatted_date = None

            # skip features without year 
            if not formatted_date:
                continue

            # extract geometry
            geometry = pa.get('geojson', {}).get('geometry')

            # debugging, print the geometry data
            if geometry is None:
                print(f"Warning: No geometry found for protected area {pa.get('name')} with ID {pa.get('id')}")
            else:
                print(f"Geometry found for protected area {pa.get('name')} with ID {pa.get('id')}")           
            
            '''
            # TO RUN TRANSFORMATION OF DATE INTO YEAR ONLY
            if date_str is None:
                year = None
            else:
                date_obj = datetime.strptime(date_str, "%d/%m/%Y")
                # extract year from datetime object
                year = date_obj.year
            '''

            # create feature with geometry and properties
            feature = {
                "type": "Feature",
                "geometry": pa.get('geojson', {}).get('geometry'),
                "properties": {
                    "id": pa.get('id'),
                    "name": pa.get('name'),
                    "original_name": pa.get('name'),
                    "wdpa_id": pa.get('id'),
                    "management_plan": pa.get('management_plan'),
                    "is_green_list": pa.get('is_green_list'),
                    "iucn_category": pa.get('iucn_category'),
                    "designation": pa.get('designation'),
                    "legal_status": pa.get('legal_status'),
                    "year": pa.get('legal_status_updated_at')
                }
            }
            # append the feature to the feature collection
            feature_collection["features"].append(feature)
        # define filename for GeoJSON file
        geojson_filename = os.path.join(response_dir, f"{country}_protected_areas.geojson")
        # convert GeoJSON data to a string
        geojson_string = json.dumps(feature_collection, indent=4) 
        # write GeoJSON string to a file
        with open(geojson_filename, 'w') as f:
            f.write(geojson_string)
        
        print(f"GeoJSON data for {country} saved to {geojson_filename}")
        
        # add the GeoJSON filename to the list
        geojson_files.append(geojson_filename)
    else:
        print(f"Error fetching data for {country}, response status code is {response}")

# define function to ensure the 'year' is formatted correctly
def format_year_attribute(geojson_file):
    with open(geojson_file, 'r') as f:
        data = json.load(f)

    for feature in data['features']:
        year_str = feature['properties'].get('year', None)
        if year_str:
            try:
                date_obj = datetime.strptime(year_str, "%d/%m/%Y")
                formatted_date = date_obj.strftime("%Y-%m-%d")
                feature['properties']['year'] = formatted_date
            except ValueError:
                feature['properties']['year'] = None
        else:
            feature['properties']['year'] = None
    
    with open(geojson_file, 'w') as f:
        json.dump(data, f, indent=4)

##### 1.3. Exporting to geopackage

In [ ]:
# define the filename for the GeoPackage
gpkg = os.path.join(response_dir, "merged_protected_areas.gpkg")
# remove GeoPackage if it already exists
if os.path.exists(gpkg):
    os.remove(gpkg)

# loop through the GeoJSON files and convert them to a geopackage
for geojson_file in geojson_files:
    # ensure the 'year' attribute is correctly formatted
    format_year_attribute(geojson_file)

    # writes layer name as the first name from geojson files
    layer_name = os.path.splitext(os.path.basename(geojson_file))[0]
    # use ogr2ogr to convert GeoJSON to GeoPackage
    subprocess.run([
        "ogr2ogr", "-f", "GPKG", "-append", "-nln", layer_name, gpkg, geojson_file
    ]) 

print(f"All GeoJSON data merged and saved to {gpkg}")

#### 2. Processing of protected areas

Data downloaded from WDPA as geopackage are processed in 4 steps:
1. Extract extent and spatial resolution of LULC data.
Redefine no data values as 0 for input LULC data.
2. Extract protected areas filtered by LULC timestamp and year of PAs establishment. As WDPA data fetching is limited by 50 features per response page, this part of code uses data downloaded not through WDPA API but through unauthorised access from WDPA website (CSV transformed into GeoPackage).
3. Rasterize protected areas (there is no way to read geodataframes by gdal_rasterize except from writing files on the disc) based on step 1.
4. Compress protected areas.

In [8]:
import geopandas as gpd
import rasterio
import os
import subprocess

# load geopackage with protected areas
gdf = gpd.read_file(r"response/pas_upd.gpkg")
# to check column names use:
# print(gdf.columns)

# define input folder
input_folder = r'lulc'
# assign output folder
output_dir = ('pas_timeseries')
# create output folder if it doesn't exist - only needed for exporting as gpkgs
os.makedirs(output_dir, exist_ok=True)

It is important to extract year stamps.

In [9]:
# list all TIFF files in input folder
tiff_files = [f for f in os.listdir(input_folder) if f.endswith('.tif')]
# extract year stamps from filenames (removes the first part before _ and the part after .)
year_stamps = [int(f.split('_')[1].split('.')[0]) for f in tiff_files]
print("Considered timestamps of LULC data are:",year_stamps)

Considered timestamps of LULC data are: [1987, 1992, 1997, 2002, 2007, 2012, 2017, 2022, 2022]


Then, extent of LULC files (minimum and maximum coordinates) is extracted.

In [10]:
# define function
def extract_ext_res(file_path):
    with rasterio.open(file_path) as src:
        extent = src.bounds
        res = src.transform[0]  # assuming the res is the same for longitude and latitude
    return extent, res

# execute function
if tiff_files:
    file_path = os.path.join(input_folder, tiff_files[0])  # choose the first TIFF file (it shouldn't matter which LULC file to extract extent because they must have the same extent)
    extent, res = extract_ext_res(file_path)
    min_x = extent.left
    max_x = extent.right
    min_y = extent.bottom
    max_y = extent.top
    
    print("Extent of LULC files")
    print("Minimum X Coordinate:", min_x)
    print("Maximum X Coordinate:", max_x)
    print("Minimum Y Coordinate:", min_y)
    print("Maximum Y Coordinate:", max_y)
    print("Spatial resolution (pixel size):", res)
else:
    print("No LULC files found in the input folder.")

# TODO - redefine null values from LULC data as 0 or something else?

Extent of LULC files
Minimum X Coordinate: 230205.0
Maximum X Coordinate: 556485.0
Minimum Y Coordinate: 4459725.0
Maximum Y Coordinate: 4777335.0
Spatial resolution (pixel size): 30.0


Protected areas should be filtered by year stamp according to the PA's establishment year.

In [9]:
# create an empty dictionary to store subsets
subsets_dict = {}
# loop through each year_stamp and create subsets
for year_stamp in year_stamps:
    # filter Geodataframe based on the year_stamp
    subset = gdf[gdf['STATUS_YR'] <= year_stamp]
    
    # store subset in the dictionary with year_stamp as key
    subsets_dict[year_stamp] = subset

    # print key-value pairs of subsets 
    print(f"Protected areas are filtered according to year stamps of LULC and PAs' establishment year: {year_stamp}")

    # ADDITIONAL BLOCK IF EXPORT TO GEOPACKAGE IS NEEDED (currently needed as rasterizing vector data is not possible with geodataframes)
    ## save filtered subset to a new GeoPackage
    subset.to_file(os.path.join(output_dir,f"pas_{year_stamp}.gpkg"), driver='GPKG')
    print(f"Filtered protected areas are written to:",os.path.join(output_dir,f"pas_{year_stamp}.gpkg"))

print ("---------------------------")

Protected areas are filtered according to year stamps of LULC and PAs' establishment year: 1987
Filtered protected areas are written to: pas_timeseries\pas_1987.gpkg
Protected areas are filtered according to year stamps of LULC and PAs' establishment year: 1992
Filtered protected areas are written to: pas_timeseries\pas_1992.gpkg
Protected areas are filtered according to year stamps of LULC and PAs' establishment year: 1997
Filtered protected areas are written to: pas_timeseries\pas_1997.gpkg
Protected areas are filtered according to year stamps of LULC and PAs' establishment year: 2002
Filtered protected areas are written to: pas_timeseries\pas_2002.gpkg
Protected areas are filtered according to year stamps of LULC and PAs' establishment year: 2007
Filtered protected areas are written to: pas_timeseries\pas_2007.gpkg
Protected areas are filtered according to year stamps of LULC and PAs' establishment year: 2012
Filtered protected areas are written to: pas_timeseries\pas_2012.gpkg
Prot

Rasterization function based on yearstamps of protected areas is launched.

In [10]:
# list all subsets of protected areas by the year of establishment
pas_yearstamps = [f for f in os.listdir(output_dir) if f.endswith('.gpkg')]
pas_yearstamp_rasters = [f.replace('.gpkg', '.tif') for f in pas_yearstamps]

# loop through each input file
for pas_yearstamp, pas_yearstamp_raster in zip(pas_yearstamps, pas_yearstamp_rasters):
    pas_yearstamp_path = os.path.join(output_dir, pas_yearstamp)
    pas_yearstamp_raster_path = os.path.join(output_dir, pas_yearstamp_raster)
    # TODO - to make paths more clear and straightforward

    # rasterize
    pas_rasterize = [
        "gdal_rasterize",
        ##"-l", "pas__merged", if you need to specify the layer
        "-burn", "100", ## assign code starting from "100" to all LULC types
        "-init", "0",
        "-tr", str(res), str(res), #spatial res from LULC data
        "-a_nodata", "-2147483647", # !DO NOT ASSIGN 0 values with non-data values as it will mask them out in raster calculator
        "-te", str(min_x), str(min_y), str(max_x), str(max_y), # minimum x, minimum y, maximum x, maximum y coordinates of LULC raster
        "-ot", "Int32",
        "-of", "GTiff",
        "-co", "COMPRESS=LZW",
        pas_yearstamp_path,
        pas_yearstamp_raster_path
        ]

    # execute rasterize command
    try:
        subprocess.run(pas_rasterize, check=True)
        print("Rasterizing of protected areas has been successfully completed for", pas_yearstamp)
    except subprocess.CalledProcessError as e:
        print(f"Error rasterizing protected areas: {e}")

Rasterizing of protected areas has been successfully completed for pas_1987.gpkg
Rasterizing of protected areas has been successfully completed for pas_1992.gpkg
Rasterizing of protected areas has been successfully completed for pas_1997.gpkg
Rasterizing of protected areas has been successfully completed for pas_2002.gpkg
Rasterizing of protected areas has been successfully completed for pas_2007.gpkg
Rasterizing of protected areas has been successfully completed for pas_2012.gpkg
Rasterizing of protected areas has been successfully completed for pas_2017.gpkg
Rasterizing of protected areas has been successfully completed for pas_2022.gpkg


##### 3. Raster calculation

LULC [enriched](/raster_sum_loop.sh) through the raster calculator (currently, external shell script):
1. Rearranging no data values as they must be considered as 0 to run raster calcualtions.
2. To sum initial LULC raster and protected areas (according to the timestamp).
3. Writing the new updated LULC map with the doubled amount of LULC codes for each timestamp (loop based on year matching in filenames).
4. Compression and assignment of null values.

##### 4. Updating landscape impedance
Impedance is reclassified by [CSV table](/reclassification.csv) and compressed (through LZW compression, not Cloud Optimised Geotiff standard to avoid any further issues in processing). Landscape impedance is required by Miramon ICT and Graphab tools both.

Let's import another set of libraries needed.

In [11]:
from osgeo import gdal
gdal.UseExceptions()
import numpy as np
import csv
import os
import subprocess

In [8]:
# specify function to reclassify LULC by mapping dictionary and obtaining impedance raster data
def reclassify_raster(input_raster, output_raster, reclass_table):
    # read reclassification table
    reclass_dict = {}
    with open(reclass_table, 'r', encoding='utf-8-sig') as csvfile:  # handle UTF-8 with BOM
        reader = csv.DictReader(csvfile)
        # initialize a flag to indicate if any row contains decimal values
        has_decimal_values = False
        
        next(reader, None) # skip headers for looping
        for row in reader:
            try:
                impedance_rounded_str = row['impedance']
                if '.' in impedance_rounded_str:  # check if impedance contains decimal values
                    has_decimal_values = True
                    break  # exit the loop if any row contains decimal values
            except ValueError:
                print("Invalid data format in reclassification table.")
            continue

        # reset file pointer to read from the beginning
        csvfile.seek(0)

        # read classification table again and define mapping for decimal and integer values
        next(reader, None) # skip headers for looping
        if has_decimal_values:
            data_type = 'Float32'
            for row in reader:
                try:
                    lulc = int(row['lulc'])
                    impedance = float(row['impedance'])
                    reclass_dict[lulc] = impedance
                except ValueError:
                    print("Invalid data format in reclassification table_2. Problematic row:", row)
                    continue
        else:
            data_type = 'Int32'
            for row in reader:
                try:
                    lulc = int(row['lulc'])
                    impedance = int(row['impedance'])
                    reclass_dict[lulc] = impedance
                except ValueError:
                    print("Invalid data format in reclassification table_3.")
                    continue
  
        if has_decimal_values:
            print("LULC impedance is characterized by decimal values.")
            # update reclassification dictionary to align nodata values with one positive value (Graphab requires positive value as no_data value)
            # assuming nodata value is 9999 (or 9999.00 if estimating decimal values)
            reclass_dict.update({-2147483647: 9999.00, -32768: 9999.00, 0: 9999.00}) # minimum value for int16, int32 and 0 are assigned with 9999.00 (nodata)
        else:
            print("LULC impedance is characterized by integer values only.")
            # update dictionary again
            reclass_dict.update({-2147483647: 9999, -32768: 9999, 0: 9999}) # minimum value for int16, int32 and 0 are assigned with 9999.00 (nodata)
    
    print ("Mapping dictionary used to classify impedance is:", reclass_dict)

    # open input raster
    dataset = gdal.Open(input_raster)
    if dataset is None:
        print("Could not open input raster.")
        return

    # get raster info
    cols = dataset.RasterXSize
    rows = dataset.RasterYSize

    # initialize output raster
    driver = gdal.GetDriverByName("GTiff")
    if has_decimal_values:
        output_dataset = driver.Create(output_raster, cols, rows, 1, gdal.GDT_Float32)
    else:
        output_dataset = driver.Create(output_raster, cols, rows, 1, gdal.GDT_Int32)
    #TODO - to add condition on Int32 if integer values are revealed
    output_dataset.SetProjection(dataset.GetProjection())
    output_dataset.SetGeoTransform(dataset.GetGeoTransform())

    # reclassify each pixel value
    input_band = dataset.GetRasterBand(1)
    output_band = output_dataset.GetRasterBand(1)
    # read the entire raster as a NumPy array
    input_data = input_band.ReadAsArray()

    # apply reclassification using dictionary mapping
    output_data = np.vectorize(reclass_dict.get)(input_data)
    output_band.WriteArray(output_data)

    '''FOR CHECKS
    print (f"input_data_shape is': {input_data.shape}")
    print (f"output_data_shape is': {output_data.shape}")
    '''

    # close datasets
    dataset = None
    output_dataset = None

    return (data_type)

if __name__ == "__main__":
    input_folder = r'lulc_pa'
    output_folder = r'impedance_pa'
    reclass_table = "reclassification.csv"
    
    # list all TIFF files in input folder
    tiff_files = [f for f in os.listdir(input_folder) if f.endswith('.tif')]
    # create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    # loop through each input file
    for tiff_file in tiff_files:
        input_raster_path = os.path.join(input_folder, tiff_file)
        print (tiff_file)
        # modify the output raster filename to ensure it's different from the input raster filename
        output_filename = "impedance_" + tiff_file
        output_raster_path = os.path.join(output_folder, output_filename)

        # call function and capture data_type for compression - Float32 or Int32
        data_type = reclassify_raster(input_raster_path, output_raster_path, reclass_table)    
        print ("Data type used to reclassify LULC as impedance is",data_type) 
        
        # compression using 9999 as nodata
        compressed_raster_path = os.path.splitext(output_raster_path)[0] + '_compr.tif'
        subprocess.run(['gdal_translate', output_raster_path, compressed_raster_path,'-a_nodata', '9999', '-ot', data_type, '-co', 'COMPRESS=LZW'])

        # as soon as gdal_translate doesn't support rewriting, we should delete non-compressed GeoTIFFs...
        os.remove(output_raster_path)
        # ...and rename compressed file in the same way as the original GeoTIFF
        os.rename(compressed_raster_path, output_raster_path)

        print("Reclassification complete for:", input_raster_path + "\n------------------------------------")


lulc_1987_pa.tif
LULC impedance is characterized by decimal values.
Mapping dictionary used to classify impedance is: {1: 4.0, 2: 1000.0, 3: 5.7, 4: 3.4, 5: 2.7, 6: 1.0, 7: 2.7, 101: 2.0, 102: 500.0, 103: 2.85, 104: 1.7, 105: 1.35, 106: 0.5, 107: 1.35, -2147483647: 9999.0, -32768: 9999.0, 0: 9999.0}


RuntimeError: TIFFAppendToStrip:Write error at scanline 1830

##### 5. Updating landscape affinity 
Landscape affinity is computed and compressed based on the math expression processing landscape impedance. By now (04/06/2024), landscape affinity is computed as a reversed value of landscape impedance but it is planned to develop it as a more flexible input to compute connectivity further. This output is required by Miramon ICT software, not Graphab.

In [12]:
impedance_dir = 'impedance_pa'
affinity_dir = 'affinity'
# create the affinity directory if it doesn't exist
if not os.path.exists(affinity_dir):
    os.makedirs(affinity_dir)

impedance_files = os.listdir(impedance_dir)
print (impedance_files)

# loop through each TIFF file in impedance_dir
for impedance_file in impedance_files:
    if impedance_file.endswith('.tif'):
        # construct full paths for impedance and affinity files
        impedance_path = os.path.join(impedance_dir, impedance_file)
        affinity_path = os.path.join(affinity_dir, impedance_file.replace('impedance', 'affinity'))

        # open impedance file
        ds = gdal.Open(impedance_path)

        if ds is None:
            print(f"Failed to open impedance file: {impedance_file}")
            continue

        # get raster band
        band = ds.GetRasterBand(1)
        # read raster band as a NumPy array
        data = band.ReadAsArray()
        # reverse values with condition (if it is 9999
        # or 0 leave it, otherwise make it reversed)
        reversed_data = np.where((data == 9999) | (data == 0), data, 1 / data)

        # write reversed data to affinity file
        driver = gdal.GetDriverByName("GTiff")
        out_ds = driver.Create(affinity_path, ds.RasterXSize, ds.RasterYSize, 1, gdal.GDT_Float32)
        out_ds.GetRasterBand(1).WriteArray(reversed_data)

        # copy georeferencing info
        out_ds.SetGeoTransform(ds.GetGeoTransform())
        out_ds.SetProjection(ds.GetProjection())

        # close files
        ds = None
        out_ds = None

        print(f"Affinity computed for: {impedance_file}")

        # compression
        compressed_raster_path = os.path.splitext(affinity_path)[0] + '_compr.tif'
        subprocess.run(['gdal_translate', affinity_path, compressed_raster_path,'-a_nodata', '9999', '-ot', 'Float32', '-co', 'COMPRESS=LZW'])
    
        # as soon as gdal_translate doesn't support rewriting, we should delete non-compressed GeoTIFFs...
        os.remove(affinity_path)
        # ...and rename COG in the same way as the original GeoTIFF
        os.rename(compressed_raster_path, affinity_path)
        print(f"Affinity file is successfully compressed.", end="\n------------------------------------------\n")

print("All LULC affinities have been successfully computed.")

['impedance_lulc_1987_pa.tif', 'impedance_lulc_1992_pa.tif', 'impedance_lulc_1997_pa.tif', 'impedance_lulc_2002_pa.tif', 'impedance_lulc_2007_pa.tif', 'impedance_lulc_2012_pa.tif', 'impedance_lulc_2017_pa.tif', 'impedance_lulc_2022_pa.tif']
Affinity computed for: impedance_lulc_1987_pa.tif
Affinity file is successfully compressed.
------------------------------------------
Affinity computed for: impedance_lulc_1992_pa.tif
Affinity file is successfully compressed.
------------------------------------------
Affinity computed for: impedance_lulc_1997_pa.tif
Affinity file is successfully compressed.
------------------------------------------
Affinity computed for: impedance_lulc_2002_pa.tif
Affinity file is successfully compressed.
------------------------------------------
Affinity computed for: impedance_lulc_2007_pa.tif
Affinity file is successfully compressed.
------------------------------------------
Affinity computed for: impedance_lulc_2012_pa.tif
Affinity file is successfully comp